In [49]:
import requests
import json
from collections import OrderedDict
import urllib

PERISCOPE_API_BASE_URL = 'https://api.periscope.tv/api/v2'
PERISCOPE_WEB_BASE_URL = 'https://www.periscope.tv'

def unescape(s):
    s = s.replace("&lt;", "<")
    s = s.replace("&gt;", ">")
    s = s.replace("&quot;", '"')
    # this has to be last:
    s = s.replace("&amp;", "&")
    return s

def get_web_public_user_session_tokens(username=None):

    user_url = '{web_base_url}/{username}'.format(web_base_url=PERISCOPE_WEB_BASE_URL, username=username)

    data_store = ""
    response = requests.get(user_url)
    if response.status_code == 200:
        html = response.text
        start = html.split('data-store="')
        end = start[1].split('"><div id="PageView"')
        data_store = json.loads(unescape(end[0]))
        
    if not data_store:
        return None

    public_tokens = data_store['SessionToken']['public']

    token_names = [
        'broadcastHistory', 'serviceToken', 'thumbnailPlaylist'
    ]

    out = {'user_id': data_store['UserBroadcastHistory']['userId']}
    for token_name in token_names:
        out[token_name] = public_tokens[token_name]['token']['session_id']

    return out

def request_api(endpoint, **params):
    url = _create_api_request_url(endpoint, **params)

    try:
        r = requests.get(url)
    except requests.exceptions.ConnectionError as e:
        raise e

    try:
        r.raise_for_status()
    except requests.exceptions.HTTPError as e:
        status_code = e.response.status_code
        if status_code == 404:
            return None
    return r.json()

def _create_api_request_url(endpoint, **params):
    params_ordered = OrderedDict(sorted(params.items()))

    url = '{base_url}/{endpoint}?{params}'.format(
        base_url=PERISCOPE_API_BASE_URL,
        endpoint=endpoint,
        params=urllib.parse.urlencode(params_ordered),
    )
    return url

def get_user_broadcast_history(username=None):
    endpoint = 'getUserBroadcastsPublic'
    session_tokens = get_web_public_user_session_tokens(username)
    if not session_tokens:
        return None

    params = {
        'user_id': session_tokens['user_id'],
        'session_id': session_tokens['broadcastHistory'],
        'all': 'true',
    }
    response = request_api(endpoint, **params)
    return response['broadcasts']

def get_broadcast_info(broadcast_id):
    endpoint = 'accessVideoPublic'
    result = request_api(endpoint, broadcast_id=broadcast_id)
    if not result:
        return None
    return result



In [51]:
username = 'CNNTurk'

broadcast_history = get_user_broadcast_history(username=username)

for broadcast_id in broadcast_history:
    bi = get_broadcast_info(broadcast_id['id'])
    if bi['type'] == "StreamTypeWeb":
        continue
    else:
        print(bi['replay_url'])

https://prod-ec-eu-central-1.video.periscope.tv/Transcoding/v1/hls/3PCz2L4VsZnLtyxlSByPft-T345-Q5GN8sIvAl0QXd1xC2VjSgPU15IFjT7agm2aj_Nu--GnShm84rN1yvjc1g/non_transcode/eu-central-1/periscope-replay-direct-prod-eu-central-1-public/master_dynamic_16848730273392643753.m3u8?type=replay
https://prod-fastly-eu-central-1.video.periscope.tv/Transcoding/v1/hls/_E8N1l8k6Llx3TLZ4lCE2WMITlkzICQVF1rmtMFdJbWQxggrLndOYe0s_-LNB-r4vNE-Xe8266a928ZPejC5tw/non_transcode/eu-central-1/periscope-replay-direct-prod-eu-central-1-public/master_dynamic_16848732227439317723.m3u8?type=replay
https://prod-fastly-eu-central-1.video.periscope.tv/Transcoding/v1/hls/inrVsEXItXLKABgprHdb0vV06vg4wo2oGT2v8kxLJ8NN51ejAhakvctz5azUDn3VT8k7vKn1xv9WVDkzoeX4uw/non_transcode/eu-central-1/periscope-replay-direct-prod-eu-central-1-public/master_dynamic_16866282108768716141.m3u8?type=replay
https://prod-fastly-eu-central-1.video.periscope.tv/Transcoding/v1/hls/LT3TQJ1buaEbDb6CiMaJ_nH3uT2He--tE-fMcwMwKsfafouGRrULTPZhoBDD0CI_XthAbgGn